In [13]:
import numpy as np
import time
from pathlib import Path
from pydrake.all import (
    BasicVector,
    Context,
    DiagramBuilder,
    LeafSystem,
    MultibodyPlant,
    RigidTransform,
    RotationMatrix,
    Simulator,
    Trajectory, 
    PiecewisePolynomial,
    PiecewisePose,
    JacobianWrtVariable,
    StartMeshcat,
    TrajectorySource,
    Integrator,
    
)

from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)
import h5py
import os
import random

In [14]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [15]:
def get_random_drawer_pose(
    r_min=0.37,      
    r_max=0.47,     
    fov_deg=90, 
    face_noise_deg=30,
    cabinet_depth=1,
    cabinet_width=1,    
):
    """
    Returns (x, y, yaw_degrees)
    """
    r = random.uniform(r_min, r_max)
    theta_rad = np.radians(random.uniform(270-fov_deg/2, 270+fov_deg/2))
    
    x = r * np.cos(theta_rad)
    y = r * np.sin(theta_rad)
    
    
    # Adjust angle b/w drawer origin is bottom right corner of cabinet
    perfect_yaw_rad = np.arctan2(-y, -x)
    yaw_rad = perfect_yaw_rad + np.radians(random.uniform(-face_noise_deg, face_noise_deg))
    
    local_center_x = cabinet_depth / 2.0
    local_center_y = cabinet_width / 2.0
    
    c_yaw = np.cos(yaw_rad)
    s_yaw = np.sin(yaw_rad)
    
    world_offset_x = local_center_x * c_yaw - local_center_y * s_yaw
    world_offset_y = local_center_x * s_yaw + local_center_y * c_yaw
    
    final_origin_x = x - world_offset_x
    final_origin_y = y - world_offset_y
    
    return final_origin_x, final_origin_y, np.degrees(yaw_rad)
    


In [27]:
def generate_scenario_string(drawer_name = "label62", **kwargs) -> str:
    drawer_urdf_path = f"{Path.cwd()}/urdf/custom/output/{drawer_name}.urdf"
    
    x, y, yaw = get_random_drawer_pose(**kwargs)
    
    scenario_string = f"""
    directives:
    # add robot
    - add_model:
        name: iiwa
        file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
        default_joint_positions:
            iiwa_joint_1: [-1.57]
            iiwa_joint_2: [0.1]
            iiwa_joint_3: [0]
            iiwa_joint_4: [-1.2]
            iiwa_joint_5: [0]
            iiwa_joint_6: [1.6]
            iiwa_joint_7: [0]
    - add_weld:
        parent: world
        child: iiwa::iiwa_link_0

    # add gripper
    - add_model:
        name: wsg
        file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
    - add_weld:
        parent: iiwa::iiwa_link_7
        child: wsg::body
        X_PC:
            translation: [0, 0, 0.09]
            rotation: !Rpy {{ deg: [90, 0, 90]}}

    # add camera mounted to world 
    - add_frame:
        name: camera0_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [250, 0, 180.0]}}
            translation: [0, 2, 1.4]
    - add_model:
        name: camera0
        file: package://manipulation/camera_box.sdf
    - add_weld:
        parent: camera0_origin
        child: camera0::base

    # add camera mounted to robot wrist
    - add_frame:
        name: camera_wrist
        X_PF:
            base_frame: iiwa::iiwa_link_7
            translation: [-0.05, 0, 0.1]   
            rotation: !Rpy {{deg: [0, 0, -90]}}
    - add_model:
        name: camera1
        file: package://manipulation/camera_box.sdf
    - add_weld:
        parent: iiwa::camera_wrist
        child: camera1::base

    # add camera mounted to world pointing top down
    - add_frame:
        name: camera2_origin
        X_PF:
            base_frame: world
            rotation: !Rpy {{ deg: [0, 180.0, 0.0]}}
            translation: [0, -0.5, 3.0]
    - add_model:
        name: camera2
        file: package://manipulation/camera_box.sdf
    - add_weld:
        parent: camera2_origin
        child: camera2::base
        
    # add random drawer
    - add_model:
        name: drawer
        file: file://{drawer_urdf_path}
    - add_frame:
        name: drawer_origin
        X_PF:
            base_frame: world
            translation: [{x}, {y}, 0]    
            rotation: !Rpy {{ deg: [0, 0, {yaw}]}}  
    - add_weld:
        parent: drawer_origin
        child: drawer::base_link
        
    cameras:
        camera0:
            name: camera0
            depth: True
            X_PB:
                base_frame: camera0::base
        camera1:
            name: camera1
            depth: True
            X_PB:
                base_frame: camera1::base
        camera2:
            name: camera2
            depth: True
            X_PB:
                base_frame: camera2::base
    
    model_drivers:
        iiwa: !IiwaDriver
            control_mode: position_only 
            hand_model_name: wsg
        wsg: !SchunkWsgDriver {{}}
    """
    return scenario_string

scenario_string = generate_scenario_string()
scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder = DiagramBuilder()
builder.AddSystem(station)
diagram = builder.Build()
context = diagram.CreateDefaultContext()

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.Initialize()

material [ 'None_NONE.006' ] not found in .mtl



In [17]:
def grasp_pose(X_WO: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
    p_OG = np.array([0.12, 0, 0])
    R_OG = RotationMatrix.MakeZRotation(np.pi/2).multiply(RotationMatrix.MakeYRotation(np.pi))
    X_OG = RigidTransform(R_OG, p_OG)
    X_WG = X_WO @ X_OG
    return X_WG

def goal_pose(X_W_Grasporiginal: RigidTransform
) -> RigidTransform:
    X_Grasporiginal_Graspgoal = RigidTransform(RotationMatrix().Identity(), [0, -0.3, 0])
    X_W_Graspgoal = X_W_Grasporiginal @ X_Grasporiginal_Graspgoal
    return X_W_Graspgoal

def approach_pose(X_WG: RigidTransform) -> RigidTransform:
    p_GGApproach = [0, -0.1, 0]
    X_GGApproach = RigidTransform(RotationMatrix(), p_GGApproach)
    X_WGApproach = X_WG @ X_GGApproach
    return X_WGApproach

def post_goal_pose(X_WGgoal: RigidTransform) -> RigidTransform:
    p_Ggoal_Gpostgoal = [0, -0.2, 0]
    X_Ggoal_Gpostgoal = RigidTransform(RotationMatrix(), p_Ggoal_Gpostgoal)
    X_WGPost = X_WGgoal @ X_Ggoal_Gpostgoal
    return X_WGPost

def make_trajectory_pose(
    X_Gs: list[RigidTransform], finger_values: np.ndarray, sample_times: list[float]
) -> tuple[Trajectory, PiecewisePolynomial]:
    poses = PiecewisePose.MakeLinear(sample_times, X_Gs)
    
    robot_velocity_trajectory = poses.MakeDerivative()
    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(sample_times, finger_values)
    return robot_velocity_trajectory, traj_wsg_command

In [18]:
class PseudoInverseController(LeafSystem):
    def __init__(self, plant: MultibodyPlant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame() 
        self._W = plant.world_frame()

        # Declaring the ports
        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa.position", 7)
        self.DeclareVectorOutputPort("iiwa.velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context: Context, output: BasicVector):
        """
        Calculates the desired joint velocities q_dot using the pseudo-inverse Jacobian.
        """
        V_WG_desired = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)

        # Using the jacobian to determine the desired joint velocity
        J_G_W = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot,
            self._G, 
            [0, 0, 0], 
            self._W, 
            self._W,
        )
        
        # Extract the values corresponding to the IIWA joints
        J_WG_iiwa = J_G_W[:, self.iiwa_start : self.iiwa_end + 1]
        J_dagger = np.linalg.pinv(J_WG_iiwa)

        v = J_dagger @ V_WG_desired
        
        output.SetFromVector(v)

In [19]:
WSG_MIN = 0.0        # fully closed width (meters)
WSG_MAX = 0.1        # fully open width (meters)

def normalize_wsg(width):
    return np.clip((width - WSG_MIN) / (WSG_MAX - WSG_MIN), 0.0, 1.0)

def unnormalize_wsg(norm_width):
    return WSG_MIN + norm_width * (WSG_MAX - WSG_MIN)

# TODO: figure out real velocity limit in simulation by closing/opening grippers
WSG_MAX_SPEED = 0.4  # meters/sec width change

def normalize_wsg_velocity(width_velocity):
    v = width_velocity / WSG_MAX_SPEED
    return np.clip(v, -1.0, 1.0)

In [20]:
def generate_trajectory(scenario_string):
    # Initializing 
    scenario = LoadScenario(data=scenario_string)
    builder = DiagramBuilder()
    station = MakeHardwareStation(scenario, meshcat=meshcat)
    builder.AddSystem(station)
    plant = station.GetSubsystemByName("plant")

    # Get initial poses of gripper and objects
    temp_context = station.CreateDefaultContext()
    temp_plant_context = plant.GetMyContextFromRoot(temp_context)
    drawer_model_instance = plant.GetModelInstanceByName("drawer")
    wsg_model = plant.GetModelInstanceByName("wsg")
    iiwa_model = plant.GetModelInstanceByName("iiwa")


    
    handle_body = plant.GetBodyByName("handle3", drawer_model_instance)
    X_W_Handle = plant.EvalBodyPoseInWorld(temp_plant_context, handle_body)

    gripper_body = plant.GetBodyByName("body", wsg_model)
    
    
    # Designing the poses (expert trajcetory)
    X_W_GripInitial = plant.EvalBodyPoseInWorld(temp_plant_context, gripper_body)
    X_W_Grasp = grasp_pose(X_W_Handle)
    X_W_Pregrasp = approach_pose(X_W_Grasp)
    X_W_Graspgoal = goal_pose(X_W_Grasp)
    X_W_Gpostgoal = post_goal_pose(X_W_Graspgoal)
    
    opened = 0.107
    closed = 0.0

    keyframes = [
        (X_W_GripInitial, opened), (X_W_Pregrasp, opened), (X_W_Grasp, opened), 
        (X_W_Grasp, closed), (X_W_Graspgoal, closed), (X_W_Graspgoal, opened), (X_W_Gpostgoal, opened), (X_W_GripInitial, opened)
    ]

    
    # # Finding and adding the trajectory
    gripper_poses = [keyframe[0] for keyframe in keyframes]
    finger_states = np.asarray([keyframe[1] for keyframe in keyframes]).reshape(1, -1)
    sample_times = [2 * i for i in range(len(gripper_poses))]
    traj_V_G, traj_wsg_command = make_trajectory_pose(gripper_poses, finger_states, sample_times)

    V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
    wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))
    controller = builder.AddSystem(PseudoInverseController(plant))
    integrator = builder.AddSystem(Integrator(7))


    # Connect Ports
    builder.Connect(V_G_source.get_output_port(0), controller.GetInputPort("V_WG"))
    builder.Connect(station.GetOutputPort("iiwa.position_measured"), controller.GetInputPort("iiwa.position"))
    builder.Connect(controller.get_output_port(0), integrator.get_input_port(0))
    
    # Crucial: We are connecting Integrator -> Station. 
    # This means the Integrator Output IS the Action.
    builder.Connect(integrator.get_output_port(0), station.GetInputPort("iiwa.position"))
    builder.Connect(wsg_source.get_output_port(0), station.GetInputPort("wsg.position"))

    diagram = builder.Build()

    # Define the simulator
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    station_context = station.GetMyContextFromRoot(context)
    integrator.set_integral_value(
        integrator.GetMyContextFromRoot(context),
        plant.GetPositions(
            plant.GetMyContextFromRoot(context),
            iiwa_model,
        ),
    )
    diagram.ForcedPublish(context)
    print(f"sanity check, simulation will run for {traj_V_G.end_time()} seconds")

    # # run simulation
    meshcat.StartRecording()
    simulator.set_target_realtime_rate(1)

    # 5. DATA LOGGING SETUP
    target_dt = 0.05 # 20 Hz
    image_data_camera0 = []
    image_data_camera1 = []
    image_data_camera2 = []
    timestamps = []
    qpos_list = []
    qvel_list = []
    action_list = []

    total_duration = traj_V_G.end_time()
    print(f"Starting collection... running for {total_duration} seconds.")

    # 6. EXECUTION LOOP
    t = 0.0
    
    while t <= total_duration:
        simulator.AdvanceTo(t)
        t = simulator.get_context().get_time()

        sim_context = simulator.get_context()
        station_context = diagram.GetSubsystemContext(station, sim_context)
        plant_context = station.GetSubsystemContext(plant, station_context)
        
        # iiwa joints
        q_iiwa = plant.GetPositions(plant_context, iiwa_model)
        v_iiwa = plant.GetVelocities(plant_context, iiwa_model)
        
        # gripper joins
        q_wsg = plant.GetPositions(plant_context, wsg_model)
        v_wsg = plant.GetVelocities(plant_context, wsg_model)
        wsg_width = normalize_wsg(q_wsg[1] - q_wsg[0])
        wsg_vel = normalize_wsg_velocity(v_wsg[1] - v_wsg[0])

        # concatenate to get exactly (7 + 1,) vectors
        qpos = np.concatenate([q_iiwa, [wsg_width]], axis=0).copy()
        qvel = np.concatenate([v_iiwa, [wsg_vel]], axis=0).copy()
        qpos_list.append(qpos)
        qvel_list.append(qvel)

        # D. Get Actions (Commands)
        # We extract what the controller/trajectory is SENDING to the robot this step.
        # For iiwa: Output of Integrator
        integrator_context = diagram.GetSubsystemContext(integrator, sim_context)
        iiwa_cmd = integrator.get_output_port(0).Eval(integrator_context)
        
        # For wsg: Output of TrajectorySource
        wsg_source_context = diagram.GetSubsystemContext(wsg_source, sim_context)
        wsg_cmd_raw = wsg_source.get_output_port(0).Eval(wsg_source_context)
        wsg_cmd_norm = normalize_wsg(wsg_cmd_raw[0]) # Normalize strictly for data consistency

        # Flatten and concatenate
        iiwa_cmd_np = np.asarray(iiwa_cmd).ravel()
        wsg_cmd_np = np.asarray([wsg_cmd_norm]).ravel()
        action = np.concatenate([iiwa_cmd_np, wsg_cmd_np], axis=0)
        action_list.append(action)

        # E. Get Images
        # Camera 0
        img0 = station.GetOutputPort("camera0.rgb_image").Eval(station_context)
        np_img0 = np.array(np.copy(img0.data)).reshape(img0.height(), img0.width(), -1)
        image_data_camera0.append(np_img0[:, :, :3].copy()) # Keep RGB only
        
        # Camera 1
        img1 = station.GetOutputPort("camera1.rgb_image").Eval(station_context)
        np_img1 = np.array(np.copy(img1.data)).reshape(img1.height(), img1.width(), -1)
        image_data_camera1.append(np_img1[:, :, :3].copy())

        # Camera 2
        img2 = station.GetOutputPort("camera2.rgb_image").Eval(station_context)
        np_img2 = np.array(np.copy(img2.data)).reshape(img2.height(), img2.width(), -1)
        image_data_camera2.append(np_img2[:, :, :3].copy())

        timestamps.append(t)
        t += target_dt

    print("Trajectory execution and logging complete.")
    meshcat.StopRecording()
    meshcat.PublishRecording()

    # 7. Return Dictionary (Standard ACT/Imitation Learning format)
    return {
        "qpos": qpos_list,
        "qvel": qvel_list,
        "action": action_list,
        "images": {
            "camera0": image_data_camera0,
            "camera1": image_data_camera1,
            "camera2": image_data_camera2
        },
        "timestamp": timestamps
    }

In [21]:
def save_trajectory_data(data: dict, episode_idx: int = 0):
    # ---------- convert buffers to numpy arrays ----------
    qpos_arr = np.stack(data['qpos'], axis=0).astype(np.float32)      # (T, nq)
    qvel_arr = np.stack(data['qvel'], axis=0).astype(np.float32)      # (T, nq)
    action_arr = np.stack(data['action'], axis=0).astype(np.float32)  # (T, act_dim)

    images0_arr = np.stack(data['images']['camera0'], axis=0).astype(np.uint8)  # (T, H, W, 3)
    images1_arr = np.stack(data['images']['camera1'], axis=0).astype(np.uint8)  # (T, H, W, 3)
    images2_arr = np.stack(data['images']['camera2'], axis=0).astype(np.uint8)  # (T, H, W, 3)

    timestamps_arr = np.array(data['timestamp'], dtype=np.float64)

    # ---------- write ACT-style HDF5 episode ----------
    h5_path = os.path.join("teleop_data/sim_open_drawer", f"episode_{episode_idx}.hdf5")
    print("Saving episode to ", h5_path)

    with h5py.File(h5_path, "w") as root:
        # mark this as simulation data (ACT uses this flag)
        root.attrs["sim"] = True

        # actions: shape (T, act_dim)
        root.create_dataset("action", data=action_arr, compression="gzip")

    # observations group
        obs_grp = root.create_group("observations")
        obs_grp.create_dataset("qpos", data=qpos_arr, compression="gzip")
        obs_grp.create_dataset("qvel", data=qvel_arr, compression="gzip")

        # optional: store timestamps (not strictly required by ACT but often useful)
        obs_grp.create_dataset("time", data=timestamps_arr)

        # images subgroup, one dataset per camera
        img_grp = obs_grp.create_group("images")
        img_grp.create_dataset("camera0", data=images0_arr, compression="gzip")
        img_grp.create_dataset("camera1", data=images1_arr, compression="gzip")
        img_grp.create_dataset("camera2", data=images2_arr, compression="gzip")

    print(f"Wrote ACT-style episode to {h5_path}")

In [28]:
num_episodes = 1

for i in range(num_episodes):
    meshcat.Delete()
    print(f"Generating episode {i}...")
    # scenario_string = generate_scenario_string(fov_deg = 0, face_noise_deg=0)
    scenario_string = generate_scenario_string()
    trajectory_data = generate_trajectory(scenario_string)
    save_trajectory_data(trajectory_data, episode_idx=i)
    print(f"Episode {i} complete.\n")
    time.sleep(1)

material [ 'None_NONE.006' ] not found in .mtl



Generating episode 0...
sanity check, simulation will run for 14.0 seconds
Starting collection... running for 14.0 seconds.
Trajectory execution and logging complete.
Saving episode to  teleop_data/sim_open_drawer/episode_0.hdf5
Wrote ACT-style episode to teleop_data/sim_open_drawer/episode_0.hdf5
Episode 0 complete.



### Watching the demonstration back

In [14]:
import imageio; 
from IPython.display import Video; 

%matplotlib inline

In [15]:
file_path = "teleop_data/sim_open_drawer/episode_0.hdf5"

f = h5py.File(file_path, "r")
data = f['observations']['images']['camera0']
frames = []
for i in range(data.shape[0]):
    frame = data[i]
    frames.append(frame)
    

imageio.mimwrite('episode_playback.mp4', data, fps=30); 
Video('episode_playback.mp4', width=480, height=360) 

ValueError: Could not find a backend to open `episode_playback.mp4`` with iomode `wI`.
Based on the extension, the following plugins might add capable backends:
  FFMPEG:  pip install imageio[ffmpeg]
  pyav:  pip install imageio[pyav]